# Этап 2. Создание рекомендательной системы для ритейл компании.
**Задачи**:
1) Проверка различных подходов в построении рекомендательной системы
2) Сочетание подходов и сравнение
3) Выбор окончательной модели и ее сохранение

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py

from scipy.sparse import csr_matrix, coo_matrix, hstack

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

from surprise import Dataset, Reader, SVD, PredictionImpossible

from annoy import AnnoyIndex

from implicit import als

from catboost import CatBoostRanker, Pool

from collections import defaultdict, Counter
from tqdm import tqdm
import random
import optuna
import gc

import warnings
warnings.filterwarnings('ignore')

Загружаем подготовленые очищенные данные по пользователям и их взаимодействиями с товарами и данные о товарах и их свойствах.

In [2]:
# датасет с пользователями и их взаимодействиями с товарами
users = pd.read_parquet('data/cleaned_events.parquet')
pd.set_option('display.max_columns', None)
users.head()

,timestamp,visitorid,event,itemid,dayofweek,is_weekend,is_holiday,hour,view_count,addtocart_count,purchase_count,conversion,first_seen,last_seen,avg_time_view,avg_time_addtocart,avg_time_transaction,total_events,items_count,purchases,session,itemevents_by_visitor,itemviews_before_purchase,time_to_purchase
0,2015-05-03 03:00:04.384,693516,addtocart,297662,6,True,False,3,63,63,0,0.0,2015-05-03 03:00:04.384,2015-09-15 19:42:17.650,56.15625,591.0,0.0,3,1,0,0.0,3,1.0,0.0
1,2015-05-03 03:00:11.289,829044,view,60987,6,True,False,3,79,79,0,0.0,2015-05-03 03:00:11.289,2015-09-14 02:38:11.012,41.21875,0.0,0.0,1,1,0,0.0,1,1.0,0.0
2,2015-05-03 03:00:13.048,652699,view,252860,6,True,False,3,248,248,0,0.0,2015-05-03 03:00:13.048,2015-09-18 02:33:52.849,13.40625,0.0,0.0,1,1,0,0.0,1,1.0,0.0
3,2015-05-03 03:00:24.154,1125936,view,33661,6,True,False,3,63,63,0,0.0,2015-05-03 03:00:24.154,2015-09-14 20:43:20.821,52.15625,0.0,0.0,1,1,0,0.0,1,1.0,0.0
4,2015-05-03 03:00:26.228,693516,view,297662,6,True,False,3,63,63,0,0.0,2015-05-03 03:00:04.384,2015-09-15 19:42:17.650,56.15625,591.0,0.0,3,1,0,0.0,3,1.0,0.0


In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2755641 entries, 0 to 2755640
Data columns (total 24 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   timestamp                  datetime64[ns]
 1   visitorid                  uint32        
 2   event                      category      
 3   itemid                     uint32        
 4   dayofweek                  uint8         
 5   is_weekend                 bool          
 6   is_holiday                 bool          
 7   hour                       uint8         
 8   view_count                 uint16        
 9   addtocart_count            uint16        
 10  purchase_count             uint8         
 11  conversion                 float16       
 12  first_seen                 datetime64[ns]
 13  last_seen                  datetime64[ns]
 14  avg_time_view              float16       
 15  avg_time_addtocart         float16       
 16  avg_time_transaction       float16  

Файл уже отсортирован по датам.

In [4]:
print(f"Временной период данных с {users['timestamp'].dt.date.min()} по {users['timestamp'].dt.date.max()}")

Временной период данных с 2015-05-03 по 2015-09-18


#### Описание признаков:
1) timestamp - время действия пользователем visitorid;
2) visitord - идентификатор пользователя;
3) event - действие (view - просмотр, addtocart - добавление в корзину, transaction - покупка);
4) itemid - идентификатор товара;
5) dayofweek - порядковый номер дня недели;
6) is_weekend - выходной True, рабочий день недели - False;
7) is_holiday - праздничные дни True, иначе - False;
8) hour - час совершения действия пользователем;
9) view_count - частота (количество) просмотров товара (itemid);
10) addtocart_count - частота (количество) добавлений товара в корзину;
11) purchase_count - частота (количество) покупок товара;
12) conversion - конверсия (purchase_count количество покупок/view_count количество просмотров);
13) first_seen - первое появление товара в данных;
14) last_seen - последняя активность товара;
15) avg_time_view - среднее время между просмотрами конкретного товара itemid;
16) avg_time_addtocart - cреднее время между добавлениями в корзину конкретного товара itemid;
17) avg_time_transaction - среднее время между покупками конкретного товара itemid;
18) total_events - активность пользователя (общее количество совершенных действий пользователем);
19) items_count - количество уникальных товаров itemid, с которым взаимодействовал пользователь;
20) purchases - лояльность пользователя (количество покупок transaction, совершенных пользователем);
21) session - cредний интервал между сессиями у пользователя, измерение в часах;
22) itemevents_by_visitor - количество действий (event) пользователя (visitorid), совершенных с конкретным товаром itemid;
23) itemviews_before_purchase - количество просмотров (view) пользователем конкретного товара перед покупкой;
24) time_to_purchase - время в часах между первым просмотром пользователем товара до его покупки.

In [5]:
# датасет с товарами и их характеристиками
items = pd.read_parquet('data/cleaned_properties.parquet')
items.head()

,timestamp,itemid,property,value,property_count,depth
0,2015-05-10 03:00:00,317951,790,32880.000,24,4
1,2015-05-10 03:00:00,422842,480,1133979,35,3
2,2015-05-10 03:00:00,310185,776,103591,25,3
3,2015-05-10 03:00:00,110973,112,679677,21,3
4,2015-05-10 03:00:00,179597,474,0,22,3


In [6]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20275902 entries, 0 to 20275901
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   timestamp       datetime64[ns]
 1   itemid          uint32        
 2   property        uint16        
 3   value           category      
 4   property_count  uint8         
 5   depth           uint8         
dtypes: category(1), datetime64[ns](1), uint16(1), uint32(1), uint8(2)
memory usage: 466.2 MB


In [7]:
print(f"Временной период данных с {items['timestamp'].dt.date.min()} по {items['timestamp'].dt.date.max()}")

Временной период данных с 2015-05-10 по 2015-09-13


Описание признаков:
1) timestamp - дата;
2) itemid - идентификатор товара;
3) property - категория товаров (уникальных 1104);
4) value - неизвестный фактор;
5) property_count - количество категорий, описывающих уникальный товар;
6) depth - глубина каждой категории (вложенности в дерево категорий)

## Подход 1. Коллаборативная фильтрация
Начнем с построения базовой модели коллаборативной фильтрации с помощью библиотеки surprise.
Основная идея коллаборативной фильтрации: похожим пользователям нравятся похожие товары, в основе лежит user-item matrix.

Давайте прежде подумаем о фильтрации данных:
1) Фильтрация пользователей и товаров даст меньше шума при обучении и модель лучше научиться рекомендовать. Как известно, метод SVD коллаборативной фильтрации зависит от качества матрицы "user-item" - чем она плотнее, тем лучше работает модель.
2) Без фильтрации:\
Когда мы применяем фильтрацию по пользователям и товарам, мы отбираем только тех пользователей и товары, которые имеют достаточно взаимодействий. Это значит, что мы не оцениваем модель на всей выборке пользователей, а только на активных пользователях и популярных товарах. В результате полученная система может показывать плохие результаты для менее активных пользователей. Наша система окажется предвзятой, ориентированной только на активных пользователей и самые популярные товары.

Мы применим минимальную фильтрацию, наша система не будет универсальной, учитывающей интересы всех пользователей, но более эффективной по качеству рекомендаций.\
Далее мы сможем дополнить данную систему контентными рекомендациями, которые будут работать для пассивных пользователей или универсальными рекомендациями самых популярных товаров.

In [8]:
df = users.copy()
print('Размер до фильтрации: ', df.shape[0])
# Подсчитаем количество уникальных товаров, с которыми взаимодействовал каждый пользователь
user_unique_items = df.groupby('visitorid')['itemid'].nunique()

# Оставим только пользователей с 4 и более уникальными товарами
min_unique_items = 4
filtered_users = user_unique_items[user_unique_items >= min_unique_items].index

# Фильтруем основной датафрейм
filtered_df = df[df['visitorid'].isin(filtered_users)]

print('Размер после фильтрации пользователей по количеству уникальным товарам: ', filtered_df.shape[0])

Размер до фильтрации:  2755641
Размер после фильтрации пользователей по количеству уникальным товарам:  789735


In [9]:
# общее число пользователей в данных
print(df.visitorid.nunique())
# размер активной части пользователей
filtered_df['visitorid'].nunique()

1407580


63294

Мы не будем строить модель только на "позитивных сигналах" (пользователь купил товар), лучше мы учтем различные факторы, которые мы сформировали для изучения взаимодействий пользователей с товарами.\
Тем самым мы постараемся включить те закомерности, которые мы увидили, и сделать модель ориентированной на различные интересы пользователей.

Нам необходимо создать взвешенный рейтинг товаров у пользователей, для этого мы используем разные веса (веса можно настраивать).

**Создание матрицы user-item.**

In [10]:
data = filtered_df[['visitorid', 'itemid', 'view_count', 'addtocart_count', 'purchase_count', 'items_count',
            'conversion', 'total_events', 'purchases', 'time_to_purchase']].copy()
data['item_rating'] = (
    0.1 * data['view_count'] +        # учет частоты просмотров товара
    0.7 * data['addtocart_count'] +   # учет добавлений товара в корзину
    1.0 * data['purchase_count'] +    # учет покупок товара
    1.0 * data['conversion'] +        # учет конверсии товара
    0.1 * data['total_events'] +      # учет общей активности пользователя
    0.5 * data['purchases'] +         # учет лояльности пользователя (количества покупок)
    0.2 * data['items_count'] +       # учет количества товаров, с которым взаимодействовал пользователь
    0.05 * data['time_to_purchase']   # учет временного интервала между первым просмотром пользователем товара до его покупки
)
# группируем по пользователям и товарам, суммируем полученный рейтинг
agg_df = data.groupby(['visitorid', 'itemid'])['item_rating'].sum().reset_index()
print("Размер данных: ", agg_df.shape[0])
agg_df.head()

Размер данных:  521872


,visitorid,itemid,item_rating
0,2,216305,946.003401
1,2,259884,304.810696
2,2,325215,284.452643
3,2,342816,184.074097
4,51,49967,99.408263


In [11]:
agg_df.item_rating.min(), agg_df.item_rating.max()

(-322.0626220703125, 128659.79653930664)

In [12]:
# масштабируем
agg_df['item_rating'] = np.log1p(agg_df['item_rating']) # логарифмируем
scaler = MinMaxScaler()
agg_df['item_rating'] = scaler.fit_transform(agg_df[['item_rating']])
agg_df.fillna(0, inplace=True)
agg_df.head()

,visitorid,itemid,item_rating
0,2,216305,0.539520
1,2,259884,0.433547
2,2,325215,0.427089
3,2,342816,0.386463
4,51,49967,0.329132


**SVD моделирование**

In [13]:
reader = Reader(rating_scale=(0, 1))  # шкала рейтинга товаров
data = Dataset.load_from_df(agg_df, reader) 

# Загружаем все данные как trainset
full_trainset = data.build_full_trainset()

# инициализируем модель SVD (сингулярное разложение матрицы)
svd_model = SVD(n_factors=300,          # n_factors — количество скрытых факторов (размерность латентного пространства)
                n_epochs=300,           # количество итераций
                lr_all=0.004,           # скорость обучения
                reg_all=0.05,           # регуляризация
                biased=True, random_state=42) 
svd_model.fit(full_trainset)

# Преобразуем trainset в testset-формат: список (uid, iid, рейтинг) для предсказания
full_testset = full_trainset.build_testset()
predictions = svd_model.test(full_testset)
predictions[:3]

[Prediction(uid=2, iid=216305, r_ui=0.5395196351944983, est=0.5251286736289278, details={'was_impossible': False}),
 Prediction(uid=2, iid=259884, r_ui=0.43354717668706433, est=0.46593538593873046, details={'was_impossible': False}),
 Prediction(uid=2, iid=325215, r_ui=0.42708851852556995, est=0.4115462576982854, details={'was_impossible': False})]

* uid: идентификатор пользователя, для которого сделано предсказание.
* iid: идентификатор товара, для которого сделано предсказание.
* r_ui: истинное значение рейтинга, которое пользователи дали товару (если было взаимодействие).
* est: значение, предсказанное моделью для данного пользователя и товара. 
* was_impossible: указывает, была ли невозможна генерация предсказания. Если значение равно False, это означает, что модель смогла сделать предсказание для этой пары пользователь-товар. Если значение True, это означает, что модель не смогла сделать предсказание (например, из-за недостатка данных для данной пары).

In [14]:
# Отбираем только те предсказания, где r_ui != 0 (иначе будет деление на 0)
filtered_preds = [(pred.r_ui, pred.est) for pred in predictions if pred.r_ui != 0]

# Разделяем на списки истинных и предсказанных значений
y_true = [true for true, pred in filtered_preds]
y_pred = [pred for true, pred in filtered_preds]

# Вычисляем MAPE
mape = mean_absolute_percentage_error(y_true, y_pred)
print(f"MAPE по всем предсказаниям SVD: {mape:.4f}")

MAPE по всем предсказаниям SVD: 0.0675


Нам необходим для вычисления Precision словарь пользователей с набором товаров, с которым взаимодействовал пользователь.

In [15]:
# Словарь: ключ - пользователь, значение - множество товаров, с которыми взаимодействовал пользователь
user_item_dict = agg_df.groupby('visitorid')['itemid'].apply(set).to_dict()
# список всех товаров 
all_item_ids = agg_df['itemid'].unique().tolist()

Создадим функцию для оценки качества подбора рекомендаций нашей моделью. Так как заказчик хочет получить систему с тремя товарами в качестве рекомендации, то нашей метрикой будет Precision@k (k=3):
$$Precision@k= \frac{\text{количество релевантных items в топ-k}}{k}$$

In [16]:
def precision_at_3_svd(user_id, svd_model, user_item_dict, all_item_ids):
    """
    Предсказывает top-3 товара с помощью SVD для пользователя и считает precision@3,
    сравнивая рекомендованные товары со всеми, с которыми пользователь взаимодействовал.
    Параметры:
    - user_id: ID пользователя
    - svd_model: обученная SVD модель
    - user_item_dict: словарь {visitorid: set(itemid)}
    - all_item_ids: список всех itemid
    Возвращает:
    - precision@3 
    """
    true_items = user_item_dict.get(user_id, set())  # товары, с которыми пользователь взаимодействовал
    
    # Предсказываем рейтинг для всех товаров
    predictions = []
    for item_id in all_item_ids:
        try:
            pred = svd_model.predict(user_id, item_id) # предсказание рейтинга
            predictions.append((item_id, pred.est))    # сохраняем только предсказанный рейтинг
        except:
            continue

    # сортируем отобранные товары по убыванию оценки и выбираем top-3
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_3 = [item for item, _ in predictions[:3]]

    # Считаем precision@3: сколько рекомендованных товаров встретилось среди всех товаров, с которыми взаимодействовал пользователь
    hits = sum([1 for item in top_3 if item in true_items])
    precision = hits / 3

    return precision

In [17]:
# сделаем расчет precision на случайной подвыборке
random_users= random.sample(list(user_item_dict.keys()), 2000)  # количество выборки можно указать любое, но считает долго

precisions = []
for user_id in tqdm(random_users):
    precision_svd = precision_at_3_svd(user_id, svd_model, user_item_dict, all_item_ids)
    if precision_svd is not None:
        precisions.append(precision_svd)

print(f"Средний Precision@3 SVD по {len(precisions)} пользователям: {np.mean(precisions):.4f}")

100%|██████████| 2000/2000 [20:11<00:00,  1.65it/s]

Средний Precision@3 SVD по 2000 пользователям: 0.0017


Создаем функцию для получения рекомендаций от SVD для конкретного пользователя.

In [18]:
def get_svd_recommendations(user_id, svd_model, user_item_dict, all_item_ids, top_n=3):
    """
    Рекомендации моделью SVD для одного пользователя.
    Параметры:
    - user_id: ID пользователя
    - svd_model: обученная модель SVD (surprise)
    - user_item_dict: словарь: пользователь -> множество просмотренных товаров
    - all_item_ids: список всех товаров
    - top_n: сколько товаров рекомендовать
    Возвращает:
    - Список top-3 itemid, предсказанных как релевантные для данного пользователя
    """
    seen_items = user_item_dict.get(user_id, set())  # товары, с которыми пользователь уже взаимодействовал

    # отбираем кандидатов — товары, которые пользователь ещё не видел
    candidate_items = list(set(all_item_ids) - seen_items)
    
    # предсказываем рейтинг для каждого кандидата с помощью модели SVD
    predictions = []
    for item in candidate_items:
        try:
            pred = svd_model.predict(user_id, item)     # предсказание рейтинга
            predictions.append((item, None, pred.est))  # сохраняем только предсказанный рейтинг
        except:
            continue  

    # сортируем отобранные товары по убыванию оценки и выбираем top-3
    predictions.sort(key=lambda x: x[2], reverse=True)
    top_items = [item for item, _, _ in predictions[:top_n]]
    
    return top_items

In [19]:
# Пример: получим рекомендации для случайного пользователя
user_id_sample = random.sample(list(user_item_dict.keys()), 1)[0]
svd_recommendations = get_svd_recommendations(user_id_sample, svd_model, user_item_dict, all_item_ids)
print(f"Рекомендованные товары для пользователя {user_id_sample} от SVD: {svd_recommendations}")

Рекомендованные товары для пользователя 696194 от SVD: [447656, 413296, 154871]


Рекомендации с помощью метода SVD показали околонулевую эффективность, поэтому мы переходим к следующему подходу.

## Подход 2. Матричная факторизация Alternating Least Squares (ALS)
Мы воспользуемся библиотекой implicit, которая требует, чтобы данные имели вид матрицы user-item, где ячейки содержат веса, отражающие интенсивность взаимодействия. Таким образом, мы возьмем созданный ранее набор agg_df.

Alternating Least Squares называют "попеременные наименьшие квадраты", так как алгоритм итеративно обучает матрицы U и I:
* сначала фиксирует I, обучает U (по методу наименьших квадратов),
* потом фиксирует U, обучает I,
* и так по очереди, пока всё не сойдётся (или до истечения количества итерации).

U - матрица пользователей (users × latent factors(скрытые характеристики)), I - матрица товаров (items × latent factors): $R = U×I$

In [20]:
agg_df.head()

,visitorid,itemid,item_rating
0,2,216305,0.539520
1,2,259884,0.433547
2,2,325215,0.427089
3,2,342816,0.386463
4,51,49967,0.329132


In [21]:
# кодируем ID пользователей и товаров
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

agg_df['user_idx'] = user_encoder.fit_transform(agg_df['visitorid'])
agg_df['item_idx'] = item_encoder.fit_transform(agg_df['itemid'])

# создаём разреженную матрицу user-item
user_item_matrix = csr_matrix((
    agg_df['item_rating'].astype(float),
    (agg_df['user_idx'], agg_df['item_idx'])
))

# инициализация и обучение ALS модели
als_model = als.AlternatingLeastSquares(
    factors=300,                # размерность скрытого пространства
    regularization=0.15,
    alpha=15,                   # вес неявных взаимодействий (implicit feedback)
    iterations=300,
    random_state=42
)
als_model.fit(user_item_matrix)

  0%|          | 0/300 [00:00<?, ?it/s]

**Функция для расчета Precision@3**

In [22]:
def precision_at_3_als(user_id, model, user_item_matrix, user_encoder, item_encoder, user_item_dict):
    """
    Делает top-3 рекомендации с помощью ALS модели и вычисляет precision@3,
    сравнивая с реальными взаимодействиями пользователя.
    Параметры:
    - user_id: идентификатор пользователя, для которого делается предсказание.
    - model: обученная модель ALS.
    - user_item_matrix: матрица взаимодействий пользователей и товаров.
    - user_encoder: кодировщик пользователей, который преобразует user_id в числовой индекс.
    - item_encoder: кодировщик товаров, который преобразует индексы товаров обратно в itemid.
    - user_item_dict: словарь, где ключ - visitorid, значение - множество товаров, с которыми пользователь взаимодействовал.
    Возвращает:
    - precision@3: доля рекомендованных товаров, которые совпали с истинными.
    """
    # Преобразуем user_id в числовой индекс для модели
    user_idx = user_encoder.transform([user_id])[0]

    # Получаем top-3 рекомендации для пользователя
    # В модели ALS каждая рекомендация возвращает пару (item_id, score, additional_info)
    recommended = model.recommend(user_idx, user_item_matrix, N=3, filter_already_liked_items=False)
    
    # Извлекаем индексы товаров из рекомендаций
    top_3 = [int(item_idx) for item_idx in recommended[0]]  # преобразуем индексы в целые числа
    
    # Преобразуем индексы товаров обратно в itemid с помощью item_encoder
    try:
        top_3_item_ids = item_encoder.inverse_transform(top_3)
    except ValueError as e:
        print(f"Ошибка при преобразовании индексов товаров: {e}")
        return None
    
    # Извлекаем товары, с которыми пользователь взаимодействовал
    true_items = user_item_dict.get(user_id, set())

    # Считаем количество совпадений
    hits = sum([1 for item in top_3_item_ids if item in true_items])

    # Вычисляем precision@3 (доля совпавших товаров из 3)
    precision = hits / 3
    return precision

Протестируем функцию на случайной подвыборке.

In [23]:
random_users = random.sample(list(user_item_dict.keys()), 10000)

precisions_als = []
for user_id in tqdm(random_users):
    precision = precision_at_3_als(user_id, als_model, user_item_matrix, user_encoder, item_encoder, user_item_dict)
    if precision is not None:
        precisions_als.append(precision)
print(f"Средний Precision@3 ALS по {len(precisions_als)} пользователям: {np.mean(precisions_als):.4f}")

100%|██████████| 10000/10000 [02:22<00:00, 70.10it/s]

Средний Precision@3 ALS по 10000 пользователям: 0.3557


Неплохо: 35% из топ-3 рекомендованных товаров для пользователей действительно являются товарами, с которыми они взаимодействовали ранее. ALS модель научилась предсказывать предпочтения пользователей даже с ограниченным списком рекомендаций.

Функция для получения рекомендаций.

In [24]:
def get_als_recommendations(user_id, als_model, user_item_matrix, user_encoder, item_encoder):
    """
    Возвращает top-3 рекомендации для одного пользователя, исключая уже взаимодействовавшие товары.
    Параметры:
    - user_id: ID пользователя в исходных данных.
    - model: обученная ALS модель.
    - user_item_matrix: разреженная матрица взаимодействий user-item.
    - user_encoder: LabelEncoder, обученный на visitorid.
    - item_encoder: LabelEncoder, обученный на itemid.
    Возвращает:
    - Список из трех itemid (товаров), рекомендованных пользователю.
    """
    try:
        # Преобразуем user_id в индекс модели
        user_idx = user_encoder.transform([user_id])[0]

        # Извлекаем строку из user_item_matrix для этого пользователя
        user_item_single = csr_matrix(user_item_matrix[user_idx:user_idx + 1, :])

        # Получаем top-3 рекомендации, исключая товары, уже знакомые пользователю
        recommended = als_model.recommend(
            user_idx,
            user_item_single,
            N=3,
            filter_already_liked_items=True  # Фильтруем товары, с которыми пользователь уже взаимодействовал
        )

        # Извлекаем индексы товаров
        recommended_item_idxs = [int(item_idx) for item_idx in recommended[0]]  # преобразуем индексы в целые числа
        
        # Переводим индексы товаров обратно в itemid
        recommended_item_ids = item_encoder.inverse_transform(recommended_item_idxs)
        
        return list(recommended_item_ids)

    except ValueError:
        # Если пользователь не был в обучающей выборке
        print(f"Пользователь {user_id} не был найден в обучающей выборке.")
        return []

In [25]:
# Пример использования
user_id = random.sample(list(user_item_dict.keys()), 1)[0] 
top3 = get_als_recommendations(user_id, als_model, user_item_matrix, user_encoder, item_encoder)
print(f"Рекомендации  ALS для пользователя {user_id}: {top3}")

Рекомендации  ALS для пользователя 712701: [257070, 259357, 132347]


Однозначно, матричная факторизация ALS предпочтительнее для нашей рекомендательной системы, чем SVD.\
Нашей целью была максимально персонализированная система, чтобы она ориентировалась на интересы каждого пользователя. Мы знаем, что для user-item матрицы важна плотность, поэтому с помощью фильтрации мы "отсеяли" пользователей и товары менее 5 активностей. А также мы постарались учесть различные факторы для создания рейтинга взаимодействия (можно проверить разные веса для сравнения).

Чтобы наша система была полноценной нам необходимо ее дополнить рекомендациями для "новичков" и пассивных пользователей с малым количеством взаимодействий + добавить разнообразие товаров (чтобы система подбирала похожие товары по категориям и включала новые/редкие).\
Для этого мы можем совместить нашу ALS с рекомендации от item-based модели, то есть использовать наиболее популярный подход - гибридную систему.

## Подход 3. Гибридная система
Такие системы позволяют делать рекомендации для новых товаров (cold start) и повышать качество рекомендаций для новых пользователей или пользователей с малым количеством взаимодействий.

**Контентная item-based модель.**

In [26]:
print('Количество объектов в датасете с товарами: ', items.shape[0])
# удалим дубликаты по товарам и категориям
items = items.drop_duplicates(subset=['itemid', 'property'])
print('Размерность после удаления дубликатов: ', items.shape[0])

Количество объектов в датасете с товарами:  20275902
Размерность после удаления дубликатов:  12003814


Признак *value* содержит какие-то идентификаторы, это могут быть id транзакций или id пользователей или что иное, информацией мы не обладаем.

In [27]:
items['value'].unique().tolist()[9:20]

['628374 827388 738621 628374 372324 703408 1182824 372324 892975 48.000 1322464 237874 914255 727274 1273256 110652 1086985 1010780 145048 834282 827388 550504 780028 85384 303286 1094068 808508 976593 210086 1297039 197566',
 '769',
 '24480.000',
 '1249027 212349 129268 726702',
 '927727',
 '1200.000 1029109',
 '1097813',
 '164766',
 '1441',
 '113745',
 '125018 1137617 424149 358393 566381 3600.000']

Предлагаю посчитать количество этих идентификаторов для каждого товара.

In [28]:
# Преобразуем колонку 'value' в строку (прежде 'category')
items['value'] = items['value'].apply(lambda x: str(x))
# разделяем с помощью split и берем длину списка
items['value_length'] = items['value'].apply(lambda x: len(x.split()))

Группируем данные по идентификатору товара itemid.

In [ ]:
grouped_items = items.groupby('itemid').agg({
    'property': lambda x: ' '.join(str(p) for p in set(x)),  # множество уникальных категорий
    'value_length': 'sum',                                   # суммируем количество идентификаторов value для товара
    'depth': 'median'                                        # медиана вложенности категории в дерево
}).reset_index()
print('Количество уникальных товаров: ', grouped_items.shape[0])
grouped_items.head()

Количество уникальных товаров:  417053


,itemid,property,value_length,depth
0,0,6 776 139 11 1036 917 790 283 159 1056 678 42 ...,40,3.0
1,1,768 0 6 776 1036 917 790 664 280 283 284 159 3...,71,3.0
2,2,641 776 778 917 790 282 283 159 678 1063 698 4...,79,3.5
3,3,1025 6 776 658 917 790 283 30 159 33 678 689 5...,36,3.0
4,4,897 6 776 917 790 283 28 159 33 678 689 698 83...,42,3.0


In [30]:
# сохраним подготовленный датасет, мы сможем его использовать для поиска товаров 
grouped_items.to_parquet('data/items.parquet')  

Так как у нас довольно много уникальных категорий для каждого товара, мы можем использовать технику TF-IDF для преобразования данного признака в числовое пространство и вычисления сходства между товарами.

In [31]:
# Масштабируем числовые признаки
scaler = MinMaxScaler()
grouped_items[['value_length', 'depth']] = scaler.fit_transform(grouped_items[['value_length', 'depth']])

# Преобразуем числовые признаки в разреженный формат
numeric_features = csr_matrix(grouped_items[['value_length', 'depth']].values)

# TF-IDF векторизация property
tfidf_vectorizer = TfidfVectorizer(max_features=600)         # мы помним, что уникальных категорий 1104
property_tfidf = tfidf_vectorizer.fit_transform(grouped_items['property'])
# Нормализуем матрицу TF-IDF
property_tfidf = normalize(property_tfidf)

# Объединяем разреженные матрицы (TF-IDF и числовые признаки)
item_features_combined = hstack([property_tfidf, numeric_features])

Annoy — библиотека от Spotify, оптимизированная для поиска ближайших соседей в больших наборах данных. Вместо точного поиска ближайших (что долго), она делает приближённый поиск, но многократно ускоряет вычисления.

Итак, c помощью AnnoyIndex построим индекс ближайших соседей по векторам, используем метрику angular — эквивалент косинусного расстояния.

In [32]:
item_features_combined.shape[1] # длина вектора

602

In [33]:
# размерность векторов
vector_size = item_features_combined.shape[1]
# Построение индекса Annoy
annoy_index = AnnoyIndex(vector_size, metric='angular')

for i in range(item_features_combined.shape[0]):
    vector = item_features_combined[i].toarray().flatten().astype(np.float32)
    annoy_index.add_item(i, vector)

# Строим и сохраняем
annoy_index.build(n_trees=20)
annoy_index.save("item_index.ann") # сохраняем

True

**Функция, которая будет возвращать наиболее похожие товары для заданного товара**

In [34]:
# Сопоставим itemid&индекс в annoy
itemid_to_index = dict(zip(grouped_items['itemid'], grouped_items.index)) # для быстрого поиска индекса в Annoy по itemid
index_to_itemid = dict(zip(grouped_items.index, grouped_items['itemid'])) # и наоборот — получение itemid по индексу из Annoy

# Функция поиска похожих товаров
def get_similar_items(itemid, top_n=3):
    if itemid not in itemid_to_index:
        return []
    item_index = itemid_to_index[itemid]
    indices = annoy_index.get_nns_by_item(item_index, top_n + 1)[1:]  # исключаем сам товар
    return [index_to_itemid[i] for i in indices] # извлекаем itemid по индексу

# Пример: похожие товары для какого-нибудь itemid
sample_item= random.sample(grouped_items['itemid'].unique().tolist(), 1)[0]
similar_items = get_similar_items(sample_item, top_n=3)
print(f"Похожие товары для item {sample_item}: {similar_items}")

Похожие товары для item 126785: [102614, 240472, 262481]


**Функция отображения похожих товаров со всеми характеристиками и проверки пересечения категорий заданного товара и рекомендованных.**

In [35]:
def get_similar_items_pretty(itemid, top_n=3):
    """
    Находит похожие товары и оценивает пересечение категорий.
    Параметры:
    - itemid: идентификатор товара, для которого ищем аналоги.
    - top_n: количество похожих товаров, которые нужно вернуть.
    Возвращает:
    - DataFrame с колонками: ['similar_to', 'itemid', 'property', 'value_length', 'depth'],
      где 'similar_to' — исходный товар, 'itemid' — похожий товар.
    Также выводит в консоль количество пересечений категорий между исходным и каждым похожим товаром.
    """
    # Получаем список itemid похожих товаров (без исходного)
    similar_ids = get_similar_items(itemid, top_n)
    # Извлекаем информацию о найденных товарах из агрегированного датафрейма
    result_df = grouped_items[grouped_items['itemid'].isin(similar_ids)].copy()
    # Добавляем колонку с ID исходного товара
    result_df['similar_to'] = itemid
    # Получаем категории (property) исходного товара как множество
    source_categories = set(
        grouped_items[grouped_items['itemid'] == itemid]['property'].iloc[0].split()
    )
    # Пересечения категорий с каждым из похожих товаров
    for sid in similar_ids:
        # Получаем категории похожего товара
        target_categories = set(
            grouped_items[grouped_items['itemid'] == sid]['property'].iloc[0].split()
        )
        # Вычисляем пересечение
        overlap = len(source_categories & target_categories)

        # Выводим количество общих категорий
        print(f"Пересечение категорий с item {sid}: {overlap} категорий")

    # Возвращаем отформатированный результат
    return result_df[['similar_to', 'itemid', 'property', 'value_length', 'depth']]

In [36]:
# пример использования
get_similar_items_pretty(sample_item)

Пересечение категорий с item 102614: 30 категорий
Пересечение категорий с item 240472: 30 категорий
Пересечение категорий с item 262481: 30 категорий


,similar_to,itemid,property,value_length,depth
91714,126785,102614,6 8 9 776 784 917 790 281 283 28 159 33 546 67...,0.329545,0.0
214894,126785,240472,6 776 9 8 784 917 790 281 283 28 159 33 546 67...,0.329545,0.0
234521,126785,262481,6 776 8 9 784 917 790 281 283 28 159 33 546 67...,0.329545,0.0


**Функция подбора похожих товаров для нового товара**

In [37]:
def find_similar_items_for_new_item(new_property, new_value_length, new_depth, top_n=3):
    """
    Ищет похожие товары для нового товара на основе property, value_length и depth.
    Параметры:
    - new_property: строка с категориями
    - new_value_length: числовое значение
    - new_depth: числовое значение
    - top_n: количество похожих товаров для возврата
    Возвращает:
    - список itemid похожих товаров
    """
    # Преобразуем property (TF-IDF)
    tfidf_vector = tfidf_vectorizer.transform([new_property])
    tfidf_vector = normalize(tfidf_vector)

    # Масштабируем числовые признаки
    numeric_vector = scaler.transform([[new_value_length, new_depth]])
    numeric_vector = csr_matrix(numeric_vector)

    # Объединяем вектор
    combined_vector = hstack([tfidf_vector, numeric_vector])
    combined_vector = combined_vector.toarray().flatten().astype(np.float32)

    # Ищем ближайших соседей через Annoy
    indices = annoy_index.get_nns_by_vector(combined_vector, top_n)
    similar_itemids = [index_to_itemid[i] for i in indices]

    return similar_itemids

In [38]:
# пример применения find_similar_items_for_new_item
sample_item_property = "888 235 65"
sample_item_value_length = 3
sample_item_depth = 3.0

similar_items = find_similar_items_for_new_item(
    new_property=sample_item_property,
    new_value_length=sample_item_value_length,
    new_depth=sample_item_depth,
    top_n=3
)
print(f"Похожие товары: {similar_items}")

Похожие товары: [431054, 115126, 15297]


### Объединение item-based подхода и модели ALS в гибридную систему.
**Описание работы гибридной рекомендательной системы:**
1. Новые пользователи (cold start), нет взаимодействий с товарами:
* Система рекомендует 2 самых популярных товара и 1 товар, похожий на самый популярный (найденный через item-based модель).
2. Пассивные пользователи (1–3 товара):
* Система выбирается случайный товар из тех, с которыми пользователь взаимодействовал.
* Для данного товара подбираются 2 похожих товара (item-based модель).
* Добавляется к списку рекомендаций 1 из топ популярных товаров.
3. Активные пользователи (4+ товара):
* Система используем модель коллаборативной фильтрации (ALS) для получения топ-3 рекомендации.
* Для каждого рекомендованного товара подбираются похожие (через item-based).
* Оба источника рекомендаций объединяются, ранжируются с учётом веса alpha (вес можно настраивать).
* Система возвращаются топ-3 товара с наибольшим итоговым весом.

In [39]:
# Подсчитаем количество уникальных товаров, с которыми взаимодействовал каждый пользователь
all_users = df.groupby('visitorid')['itemid'].nunique()
# Оставим только пользователей с менее чем 4 уникальных товаров
min_unique_items = 4
# пассивные пользователи
passive_users = set(all_users[all_users < min_unique_items].index.tolist())
print('Количество пользователей с менее 4 уникальными товарами: ', len(passive_users))

# Словарь для пассивных пользователей: ключ - пользователь, значение - множество товаров, с которыми взаимодействовал пользователь
user_item_for_passive = df[df['visitorid'].isin(passive_users)].groupby('visitorid')['itemid'].apply(set).to_dict()

Количество пользователей с менее 4 уникальными товарами:  1344286


In [40]:
def get_hybrid_recommendations(user_id, user_item_dict, user_item_for_passive, annoy_index, top_n=3, alpha=0.7):
    
    # Получаем топ-2 популярных товара
    top_popular_items = agg_df['itemid'].value_counts().head(2).index.tolist()

    # Получаем один товар, похожий на самый популярный
    similar_item_for_popular = get_similar_items(top_popular_items[0], top_n=1)
    if not isinstance(similar_item_for_popular, list):
        similar_item_for_popular = list(similar_item_for_popular)
    
    # рекомендации для пассивных пользователей (с 1-3 товарами)
    if user_id in user_item_for_passive:
        items = user_item_for_passive.get(user_id, set())
        items = list(items)
        if len(items) > 0:
            random_item = random.sample(items, 1)[0]
            sim_items = get_similar_items(random_item, top_n=2)
            if not isinstance(sim_items, list):
                sim_items = list(sim_items)
            recommendations_for_passive_user = sim_items + [top_popular_items[0]]
            return recommendations_for_passive_user
        else:
            # пользователь в списке пассивных, но без взаимодействий
            return top_popular_items + similar_item_for_popular
    
    # Если пользователя нет в базе, рекомендуем топовые товары + похожие товары для популярных
    if user_id not in user_item_for_passive and user_id not in user_item_dict:
        recommendations_new = top_popular_items + similar_item_for_popular
        return recommendations_new
    
    # рекомендации для активных пользователей (4+ товара)
    if user_id in user_item_dict:
        als_recommendations = get_als_recommendations(user_id, als_model, user_item_matrix, user_encoder, item_encoder)

        content_recommendations = []
        for recommend in als_recommendations:
            similar_items = get_similar_items(recommend, top_n=2)
            if not isinstance(similar_items, list):
                similar_items = list(similar_items)
            content_recommendations.extend(similar_items)
        
        # добавляем веса
        weighted_scores = Counter()
        for i, item in enumerate(als_recommendations):
            weighted_scores[item] += alpha * (top_n - i)
        for i, item in enumerate(content_recommendations):
            weighted_scores[item] += (1 - alpha) * (top_n - i)

        ranked_items = [item for item, _ in weighted_scores.most_common()]
        final_recommendations = ranked_items[:top_n]
        return final_recommendations

In [41]:
# Пример использования
user_id_example = random.sample(users['visitorid'].unique().tolist(), 1)[0]
hybrid_recommend = get_hybrid_recommendations(user_id_example, user_item_dict, user_item_for_passive, annoy_index, top_n=3, alpha=0.7)
print(f"Гибридные рекомендации для пользователя {user_id_example}: {hybrid_recommend}")

Гибридные рекомендации для пользователя 76706: [8436, 14188, 257040]


Функция расчета precision@3 для гибридной системы, но мы ее применим для нашей активной базы пользователей.\
Мы получили хорошее качество от ALS для них и хотим проверить, насколько оно отклонится с подключением item-based модели для добавления разнообразия новыми товарами.

In [42]:
def hybrid_precision_for_active_users(user_id, als_model, user_item_dict, user_item_matrix, 
                                      user_encoder, item_encoder, annoy_index, top_n=3, alpha=0.7):
    """
    Оценивает Precision@3 для гибридной рекомендательной системы только на активных пользователях.
    Параметры:
    - user_id: id пользователя
    - user_item_dict: словарь {visitorid: set(itemid)}, товары, с которыми взаимодействовал пользователь
    - user_item_matrix: разреженная матрица взаимодействий user-item
    - user_encoder, item_encoder: кодировщики
    - annoy_index: индекс Annoy для поиска похожих товаров
    - top_n: количество рекомендаций
    - alpha: вес рекомендаций от ALS (0.0–1.0)
    Возвращает:
    - средний precision@3 по всем пользователям
    """
    # Преобразуем user_id в числовой индекс для модели
    user_idx = user_encoder.transform([user_id])[0]

    # Получаем top-3 рекомендации для пользователя от ALS 
    recommended = als_model.recommend(user_idx, user_item_matrix, N=3, filter_already_liked_items=False) # включаем все товары
    
    # Извлекаем индексы товаров из рекомендаций
    top_3_als = [int(item_idx) for item_idx in recommended[0]]  # преобразуем индексы в целые числа
    
    # Преобразуем индексы товаров обратно в itemid с помощью item_encoder
    try:
        top_3_item_ids = item_encoder.inverse_transform(top_3_als)
    except ValueError as e:
        print(f"Ошибка при преобразовании индексов товаров: {e}")
        return None
    
    # добавляем контентные рекомендации
    content_recommendations = []
    for recommend in top_3_item_ids:
        similar_items = get_similar_items(recommend, top_n=2)
        if not isinstance(similar_items, list):
            similar_items = list(similar_items)
        content_recommendations.extend(similar_items)

    weighted_scores = Counter()
    for i, item in enumerate(top_3_item_ids):
        weighted_scores[item] += alpha * (top_n - i)
    for i, item in enumerate(content_recommendations):
        weighted_scores[item] += (1 - alpha) * (top_n - i)
    
    # ранжируем
    ranked_items = list(dict.fromkeys([item for item, _ in weighted_scores.most_common()]))
    top_items = ranked_items[:top_n]

    # Извлекаем товары, с которыми пользователь взаимодействовал
    true_items = user_item_dict.get(user_id, set())
    
    # Считаем количество совпадений
    hits = sum([1 for item in top_items if item in true_items])

    # Вычисляем precision@3 (доля совпавших товаров из 3)
    precision = hits / 3
    
    return precision

In [43]:
precisions_hybrid = []
for user_id in tqdm(random_users): # та же выборка, что тестировалась для модели ALS
    precision = hybrid_precision_for_active_users(user_id, als_model, user_item_dict, user_item_matrix, 
                                      user_encoder, item_encoder, annoy_index, top_n=3, alpha=0.8)
    if precision is not None:
        precisions_hybrid.append(precision)
print(f"Средний Precision@3 Hybrid System по {len(precisions_hybrid)} пользователям: {np.mean(precisions_hybrid):.4f}")

100%|██████████| 10000/10000 [12:05<00:00, 13.79it/s]

Средний Precision@3 Hybrid System по 10000 пользователям: 0.3517


Гибридная рекомендательная система направлена на разнообразие рекомендаций и учитывает следующие ключевые аргументы:
1) Охват всех пользователей и товаров: это позволяет системе быть универсальной и давать рекомендации не только для активных пользователей, но и для тех, кто только начинает взаимодействовать с платформой.
2) Учет популярности товаров: популярные товары, которые активно просматриваются или покупаются, получают более высокие приоритеты в рекомендациях, что помогает предложить пользователям наиболее востребованные товары.
3) Учет активности клиентов: для новых пользователей, а также тех, кто редко взаимодействует с системой, могут быть рекомендованы товары на основе общей популярности, а не исторических предпочтений.
4) Включение разнообразия в товарах: рекомендации товаров из похожих категорий.
Таким образом, гибридная система не только повышает точность рекомендаций, но и помогает эффективно работать с разнообразными сегментами пользователей, включая тех, кто только начинает использовать систему.

## Подход 4. Ранжирование CatBoostRanker 
CatBoostRanker решает задачу ранжирования, где для каждой группы объектов (в нашем случае, для каждого пользователя) необходимо определить порядок объектов (товары), основываясь на их характеристиках.

In [44]:
# мы возьмем отфильтрованные данные (пользователи с 4+ товарами) для обучения модели
data = filtered_df.copy()

In [45]:
# Создание целевой переменной для ранжирования
event_priority = {'view': 0, 'addtocart': 1, 'transaction': 2}   # чем выше значение, тем выше "ценность" события
data['label'] = data['event'].map(event_priority)

# Удаление ненужных признаков 
data = data.drop(columns=['first_seen', 'last_seen', 'event'])

# Создание списка категориальных признаков 
cat_features = ['visitorid', 'itemid', 'dayofweek', 'is_weekend', 'is_holiday', 'hour']
for col in cat_features:
    data[col] = data[col].astype('category') 

# Сортировка по времени
data = data.sort_values('timestamp')
data = data.drop(columns=['timestamp'])  # timestamp больше не нужен

# Определение списка признаков
features = [col for col in data.columns if col not in ['label']]
data.to_parquet('data/df_ranker.parquet') # сохраняем данные к том виде, как будем подавать для модели в рекомендательной системе

# Разделение данных: 90% - обучение, 10% - тест
train_df, test_df = train_test_split(data, test_size=0.1, shuffle=False) # сохраняем последовательность данных
train_df.head()

,visitorid,itemid,dayofweek,is_weekend,is_holiday,hour,view_count,addtocart_count,purchase_count,conversion,avg_time_view,avg_time_addtocart,avg_time_transaction,total_events,items_count,purchases,session,itemevents_by_visitor,itemviews_before_purchase,time_to_purchase,label
11,1399056,197200,6,True,False,3,32,32,0,0.000000,102.62500,0.000000,0.0,6,5,0,0.029999,2,2.0,0.0,0
17,990356,161722,6,True,False,3,9,9,0,0.000000,386.50000,0.000000,0.0,14,12,0,195.375000,1,1.0,0.0,0
24,90447,242145,6,True,False,3,59,59,0,0.000000,56.75000,0.000000,0.0,7,6,0,372.250000,1,1.0,0.0,0
26,979664,338222,6,True,False,3,53,53,0,0.000000,52.09375,1.870117,0.0,11,9,0,67.500000,2,1.0,0.0,1
29,1399056,48743,6,True,False,3,174,174,1,0.005749,16.06250,166.750000,0.0,6,5,0,0.029999,1,1.0,0.0,0


In [46]:
# Сортировка по пользователю перед ранжированием 
train_df = train_df.sort_values(by='visitorid') # CatBoost требует, чтобы объекты одной группы (visitorid) шли подряд
test_df = test_df.sort_values(by='visitorid')

# Подготовка пулов для CatBoostRanker
train_pool = Pool(
    data=train_df[features],
    label=train_df['label'],
    group_id=train_df['visitorid'],
    cat_features=cat_features
)
test_pool = Pool(
    data=test_df[features],
    label=test_df['label'],
    group_id=test_df['visitorid'],
    cat_features=cat_features
)

Для поиска оптимальных параметров для модели мы не будем использовать все данные, это очень затратно по ресурсам.
Мы сделаем мини-наборы, подберем гиперпараметры, а затем обучим итогую модель на них.

In [47]:
# Используем первые 15% данных из train_df для обучения
train_size = int(0.15*len(train_df))  
train_df_sampled = train_df.iloc[:train_size]  # первые 15% данных по порядку

# Для валидации берём следующие данные в 4 раза меньше по размеру
valid_size = train_size // 4
valid_df_sampled = train_df.iloc[train_size:train_size+valid_size]

print(train_df_sampled.shape, valid_df_sampled.shape)

train_pool_sampled = Pool(
    data=train_df_sampled[features],
    label=train_df_sampled['label'],
    group_id=train_df_sampled['visitorid'],
    cat_features=cat_features
)
valid_pool_sampled = Pool(
    data=valid_df_sampled[features],
    label=valid_df_sampled['label'],
    group_id=valid_df_sampled['visitorid'],
    cat_features=cat_features
)

(106614, 21) (26653, 21)


Для подбора гиперпараметров обратимся к оптимизатору optuna.

In [48]:
# Функция для оптимизации гиперпараметров
def objective(trial):
    # Определяем гиперпараметры для оптимизации
    params = {
        'iterations': trial.suggest_categorical('iterations', [400, 500, 600]),  # Количество итераций
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.05, 0.1),  # Скорость обучения
        'depth': trial.suggest_int('depth', 7, 11),  # Глубина деревьев
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 2, 6),  # Регуляризация
        'loss_function': 'YetiRank',  # Функция потерь
        'eval_metric': 'NDCG',  # Метрика для оценки
        'verbose': 0,
        'random_seed': 42,
        'task_type': 'CPU',
        'thread_count': 4,
        'use_best_model': True
    }
        
    # Инициализация модели с подобранными параметрами
    ranker = CatBoostRanker(**params)
    ranker.fit(train_pool_sampled, eval_set=valid_pool_sampled)

    # Получаем все метрики с помощью get_evals_result
    evals_result = ranker.get_evals_result()
        
    # Извлекаем последнее значение NDCG для валидационной выборки
    ndcg_score = evals_result['validation']['NDCG:type=Base'][-1]
        
    # Очистка памяти
    del ranker
    gc.collect()

    # Возвращаем NDCG
    return ndcg_score

# Инициализация и запуск оптимизации гиперпараметров
study = optuna.create_study(direction='maximize')  # максимизируем NDCG
study.optimize(objective, n_trials=10)  # число проб для подбора гиперпараметров, чем больше, тем лучше

# Результаты оптимизации
print(f"Наилучшие гиперпараметры: {study.best_trial.params}")

[I 2025-04-14 16:37:09,405] A new study created in memory with name: no-name-5bb8a13f-c6a0-4048-ba3a-2f12e76460b6
[I 2025-04-14 16:40:04,224] Trial 0 finished with value: 0.9401404663054221 and parameters: {'iterations': 500, 'learning_rate': 0.05091573520439335, 'depth': 9, 'l2_leaf_reg': 6}. Best is trial 0 with value: 0.9401404663054221.
[I 2025-04-14 16:43:30,402] Trial 1 finished with value: 0.9406198411358683 and parameters: {'iterations': 600, 'learning_rate': 0.08589047171684033, 'depth': 8, 'l2_leaf_reg': 2}. Best is trial 1 with value: 0.9406198411358683.
[I 2025-04-14 16:47:30,632] Trial 2 finished with value: 0.9398285902061356 and parameters: {'iterations': 500, 'learning_rate': 0.06799327450427634, 'depth': 10, 'l2_leaf_reg': 6}. Best is trial 1 with value: 0.9406198411358683.
[I 2025-04-14 16:51:40,877] Trial 3 finished with value: 0.9411173982244087 and parameters: {'iterations': 600, 'learning_rate': 0.07088652441295533, 'depth': 11, 'l2_leaf_reg': 5}. Best is trial 3 

Наилучшие гиперпараметры: {'iterations': 600, 'learning_rate': 0.07088652441295533, 'depth': 11, 'l2_leaf_reg': 5}


In [49]:
opt_params = study.best_trial.params
# Создание модели с подобранными гиперпараметрами и обучение
ranker = CatBoostRanker(
    iterations=opt_params['iterations'],
    learning_rate=opt_params['learning_rate'],
    depth=opt_params['depth'],
    l2_leaf_reg=opt_params['l2_leaf_reg'],
    loss_function='YetiRank',   # функция потерь для ранжирования
    eval_metric='NDCG',
    verbose=0,
    random_seed=42,
    use_best_model=True,
    thread_count = 4,
    task_type='CPU'  # если есть возможность, лучше сменить на 'GPU'
)
ranker.fit(train_pool, eval_set=test_pool)

In [50]:
# Предсказания
train_df['preds'] = ranker.predict(train_pool)
test_df['preds'] = ranker.predict(test_pool)
test_df[['visitorid', 'itemid', 'preds']].head(10)

,visitorid,itemid,preds
2537211,51,358388,-1.754923
2537225,51,198762,-1.439822
2537260,51,429304,-1.510127
2537319,51,198762,-1.439822
2537341,51,49967,-1.710541
2537357,51,279059,-2.007947
2724929,54,283115,-1.162743
2724797,54,38965,-1.157952
2724839,54,319680,-1.144212
2738657,54,249114,-1.928771


Посмотрим Precision сначала на тренировочном наборе.

In [51]:
# Группируем пользователей в train_df
true_items_train = train_df.groupby('visitorid')['itemid'].apply(set)

# Фильтруем train_df для пользователей, которые есть в true_items_train
train_df_filtered = train_df[train_df['visitorid'].isin(true_items_train.index)]

# Топ-3 предсказанных товаров на основе ranker для train_df
top_n = 3
top_preds_train = (
    train_df_filtered.sort_values(by=['visitorid', 'preds'], ascending=[True, False])
    .groupby('visitorid').head(top_n)
    .groupby('visitorid')['itemid'].apply(list)
)

# Подсчёт precision@3 для train_df
precision_scores_train = []
for user_id in tqdm(train_df_filtered['visitorid'].unique()):  # Используем уникальные visitorid
    pred_items = top_preds_train.get(user_id, [])  # Получаем предсказанные товары для пользователя
    actual_items = true_items_train.get(user_id, set())  # Получаем реальные товары для пользователя
    hits = sum([1 for item in pred_items if item in actual_items])  # Подсчёт совпадений
    precision_scores_train.append(hits / top_n)

# Проверим размер
print("Количество пользователей в precision_scores_train: ", len(precision_scores_train))

# Средний precision@3 для train_df
precision_at_3_catboost_train = sum(precision_scores_train) / len(precision_scores_train)
print(f"Precision@3 от CatBoostRanker на обучающей выборке : {precision_at_3_catboost_train:.4f}")

100%|██████████| 58724/58724 [00:00<00:00, 83909.26it/s]

Количество пользователей в precision_scores_train:  58724
Precision@3 от CatBoostRanker на обучающей выборке : 0.9900


Давайте сравним результат с Precision на тестовой выборке.

In [52]:
# Словарь товаров для каждого пользователя, с которыми он взаимодействовался в test_df
true_items_test = test_df.groupby('visitorid')['itemid'].apply(set)
test_df_filtered = test_df[test_df['visitorid'].isin(true_items_test.index)]

# Топ-3 предсказанных товаров на основе модели
top_n = 3
top_preds_test = (
    test_df_filtered.sort_values(by=['visitorid', 'preds'], ascending=[True, False])
    .groupby('visitorid').head(top_n)
    .groupby('visitorid')['itemid'].apply(list)
)

# Подсчёт precision@3
precision_scores_test = []
for user_id in tqdm(test_df_filtered['visitorid'].unique()):  # Используем уникальные visitorid
    pred_items = top_preds_test.get(user_id, [])  # Получаем предсказанные товары для пользователя
    actual_items = true_items_test.get(user_id, set())  # Получаем реальные товары для пользователя
    hits = sum([1 for item in pred_items if item in actual_items])  # Подсчёт совпадений
    precision_scores_test.append(hits / top_n)

# Проверим размер
print("Количество пользователей в precision_scores_test: ", len(precision_scores_test))


# Средний precision@3
precision_at_3_catboost_test = sum(precision_scores_test) / len(precision_scores_test)
print(f"Precision@3 от CatBoostRanker на тестовой выборке : {precision_at_3_catboost_test:.4f}")

100%|██████████| 10394/10394 [00:00<00:00, 89483.21it/s]

Количество пользователей в precision_scores_test:  10394
Precision@3 от CatBoostRanker на тестовой выборке : 0.8311


In [53]:
# Выбор одного пользователя из тестового набора
sample_id = random.choice(test_df['visitorid'].unique().tolist())
sample_user= test_df[test_df['visitorid'] == sample_id]

# Предсказание для выбранного пользователя
sample_user['score'] = ranker.predict(sample_user[features])

# Сортировка по предсказанным оценкам и выбор top-3 товаров
top_items = sample_user.sort_values('score', ascending=False)['itemid'].head(3).tolist()
print(f"Top-3 рекомендации от CatBoostRanker для пользователя {sample_id}: {top_items}")

Top-3 рекомендации от CatBoostRanker для пользователя 972387: [461686, 461686, 461686]


In [54]:
# отобразим на графике значения метрики precision двух лучших подходов
models = ['ALS', 'CatBoostRanker']
scores = [np.mean(precisions_als), precision_at_3_catboost_test]

fig = go.Figure([
    go.Bar(x=models, y=scores, marker_color=['blue', 'green'])
])
fig.update_layout(
    title='Сравнение моделей по Precision@3',
    yaxis=dict(title='Precision@3'),
    xaxis=dict(title='Модель'),
    template='plotly',
    autosize=True, margin=dict(t=40, b=30, l=30, r=10),
    width=400, height=400
)
fig.show()

In [55]:
# сохраним модель
ranker.save_model('catboost_ranker.bin')

### <center>Заключение</center>
Самым лучшим алгоритмом для формирования рекомендаций для активной аудитории пользователей (63294 человека), история которых начинается от 4 действий на платформе, оказался CatBoostRanker, его рекомендации на тестовой выборке совпали примерно на 83% с товарами, которые пользователь выбрал сам. 

Но в текущей базе данных имеется 1,407,580 пользователей, а значит, рекомендательной системой, которая будет учитывать как активную часть, так и пассивную (что является абсолютным большинством 1,344,286) или новых пользователей лучше выбрать гибридную, которая будет включать как рекомендации от ранжировщика catboost, так и рекомендации самых популярных товаров для "новичков", а также подбор похожих товаров по категориям для пассивных пользователей с небольшой историей (1-3 действия).

Мы строили и тестировали модели с ориентиром на любые действия от пользователей, а не только на покупки, потому что доля транзакций составляет менее 1% от всех событий, поэтому было важно учитывать просмотры, но с меньшим весом.

Следующим шагом будет создание веб-сервиса гибридной рекомендательной системы. Настройка сбора метрик и мониторинга за системой.
